In [1]:
import cv2
import os
import mediapipe as mp
import imutils

In [2]:
Datos = []
Nombres = []
Cargos = []
Edades = []
entrar_salir = [False,False]

datosPath = 'C:/Users/uriel/OneDrive - Universidad de Guanajuato/Documentos/8vo semestre/Forte/Datos'
peopleList = os.listdir(datosPath)
for person in peopleList: 
    filePathPerson = datosPath + '/' + person
    with open(filePathPerson, "r") as file:
        Datos.append(file.readlines())


for Dato in Datos:
    aux = Dato[0].split()
    Nombres.append(aux[1])
    aux = Dato[2].split()
    Cargos.append(aux[1])
    aux = Dato[4].split()
    Edades.append(aux[1])
    
print(Nombres)
print(Cargos)
print(Edades)

['Mariel', 'Uriel']
['Estudiante', 'Practicante']
['14', '21']


In [18]:
#face_recognizer = cv2.face.EigenFaceRecognizer_create()
#face_recognizer = cv2.face.FisherFaceRecognizer_create()
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

# Leyendo el modelo
#face_recognizer.read('modeloEigenFace.xml')
#face_recognizer.read('modeloFisherFace_UyM.xml')
face_recognizer.read('modeloLBPHFace_UyM.xml')

In [20]:
mp_face_detection = mp.solutions.face_detection # Es el detector de rostros
mp_drawing = mp.solutions.drawing_utils # Esto es para colocar el recuadro y los puntos
cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
mp_face_mesh = mp.solutions.face_mesh

with mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    min_detection_confidence=0.5) as face_mesh:
    with mp_face_detection.FaceDetection(  # Gestor de contexto
        min_detection_confidence=0.6) as face_detection:

        while True:
            ret, frame = cap.read() # Comenzamos a leer el video
            if ret == False: break # Verificamos si se esta leyendo de forma correcta
            frame = imutils.resize(frame, width=640)
            frame = cv2.flip(frame, 1)
            aux_frame = frame.copy()
            height, width, _ = frame.shape # Obtenemos las dimensipones de la imagen
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # cv2 lo lee como BGR y lo convertimos a RGB       
            results = face_detection.process(frame_rgb) # Obtenemos las cordenadas de la cara y de los puntos de interes
            malla = face_mesh.process(frame_rgb)
            k = cv2.waitKey(1) & 0xFF
            
            
            if results.detections is not None: # Si detecta un rostro entra aqui 
                if malla.multi_face_landmarks is not None:
                    for face_landmarks in malla.multi_face_landmarks:
                        
                        longitud = []
                        for detection in results.detections:
                            w = int(detection.location_data.relative_bounding_box.width * width)
                            h = int(detection.location_data.relative_bounding_box.height * height)
                            longitud.append(w+h)

                        id_min = longitud.index(max(longitud))  
                        face = results.detections[id_min]
                        x = int(face.location_data.relative_bounding_box.xmin * width)
                        y = int(face.location_data.relative_bounding_box.ymin * height)    

                        # Recortamos el rostro y reajustamos el tamaño
                        rostro = aux_frame[y:y+h,x:x+w]
                        try:
                            rostro = cv2.resize(rostro,(150,150), interpolation=cv2.INTER_CUBIC)
                            rostro = cv2.cvtColor(rostro, cv2.COLOR_BGR2GRAY)
                            label = face_recognizer.predict(rostro)
                            cv2.putText(frame,'{}'.format(label),(x,y-5),1,1.3,(255,255,0),1,cv2.LINE_AA)
                            coord_x_text = x-w-45

                            if label[1] < 110:
                                cv2.putText(frame,'Nombre:{}'.format(Nombres[label[0]]),(coord_x_text,y-65),2,1.1,(255,0,0),1,cv2.LINE_AA)
                                cv2.putText(frame,'Cargo:{}'.format(Cargos[label[0]]),(coord_x_text,y-30),1,2.1,(255,0,0),1,cv2.LINE_AA)
                                cv2.putText(frame,'Edad:{}'.format(Edades[label[0]]),(coord_x_text,y+5),1,2.1,(255,0,0),1,cv2.LINE_AA)   
                                #cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)   
                                mp_drawing.draw_landmarks(frame, face_landmarks,
                                    mp_face_mesh.FACEMESH_CONTOURS , #FACEMESH_TESSELATION FACEMESH_CONTOURS  FACE_CONNECTIONS
                                    mp_drawing.DrawingSpec(color=(0,255, 0), thickness=1, circle_radius=1),
                                    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1))

                            else:
                                cv2.putText(frame,'Desconocido',(coord_x_text,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
                                #cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 3)
                                mp_drawing.draw_landmarks(frame, face_landmarks,
                                    mp_face_mesh.FACEMESH_CONTOURS , #FACEMESH_TESSELATION FACEMESH_CONTOURS  FACE_CONNECTIONS
                                    mp_drawing.DrawingSpec(color=(0,0, 255), thickness=1, circle_radius=1),
                                    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1))

                        except:
                            pass

            cv2.imshow("frame", frame) # Muestra lo que esta detectando 
            if k == 27: break# Si tecleamos la tecla esc se cancela o si llega a 300
cap.release()
cv2.destroyAllWindows()